In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI, ChatOpenAI

llm=OpenAI(
    api_key="sk-zk282c91e7660ee3aefc73a38060adc99596725965513925",
    base_url="https://api.zhizengzeng.com/v1/",
    model="gpt-3.5-turbo"
)

chat=ChatOpenAI(
    api_key="sk-zk282c91e7660ee3aefc73a38060adc99596725965513925",
    base_url="https://api.zhizengzeng.com/v1/",
    model="gpt-3.5-turbo"
)


In [3]:
ans=chat.invoke("帮我写一个笑话")

In [4]:
print(ans.content)

为什么书本不喜欢去派对？

因为他们总是被人翻来翻去，而且还要忍受不断的注释和批注！


In [5]:
ans=llm.client.create(model="gpt-3.5-turbo-instruct",prompt="帮我写一个关于计算机的笑话")

In [6]:
ans.choices[0].text

'\n\n老师：同学们，今天我们来讲一'

In [14]:
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain_openai import OpenAIEmbeddings
import os

api_key="sk-zk282c91e7660ee3aefc73a38060adc99596725965513925"
base_url="https://api.zhizengzeng.com/v1/"

os.environ['http_proxy']='http://127.0.0.1:8080'
os.environ['https_proxy']='http://127.0.0.1:8080'

embedding = OpenAIEmbeddings(api_key=api_key,base_url=base_url)


# 加载文档并禁用 SSL 验证
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
loader.requests_kwargs = {'verify': False}

# 创建向量索引
index_creator = VectorstoreIndexCreator(embedding=embedding)
index = index_creator.from_loaders([loader])

# 测试查询
query = "解释一下文章的主要内容。"
results = index.query(query)
print("查询结果:", results)

/Applications/anaconda/anaconda3/lib/python3.12/site-packages/langchain/indexes/vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(
/Applications/anaconda/anaconda3/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


SSLError: HTTPSConnectionPool(host='openaipublic.blob.core.windows.net', port=443): Max retries exceeded with url: /encodings/cl100k_base.tiktoken (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

In [ ]:
from langchain.embeddings.base import Embeddings
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
import requests
import os

# 配置代理
os.environ['http_proxy'] = 'http://127.0.0.1:8080'
os.environ['https_proxy'] = 'http://127.0.0.1:8080'

# 自定义嵌入类
class CustomEmbedding(Embeddings):
    def __init__(self, api_key, base_url):
        self.api_key = api_key
        self.base_url = base_url

    def embed_documents(self, texts):
        return [self.embed_text(text) for text in texts]

    def embed_query(self, text):
        return self.embed_text(text)

    def embed_text(self, text):
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        data = {"input": text}

        # 请求自定义的嵌入 API
        response = requests.post(f"{self.base_url}/embeddings", headers=headers, json=data)
        if response.status_code == 200:
            return response.json().get("embedding", [])
        else:
            print("请求嵌入失败，状态码:", response.status_code)
            return []

# 使用自定义嵌入模型
api_key = "sk-zk282c91e7660ee3aefc73a38060adc99596725965513925"
base_url = "https://api.zhizengzeng.com/v1"
embedding = CustomEmbedding(api_key=api_key, base_url=base_url)


# 忽略 SSL 证书验证
requests.packages.urllib3.disable_warnings()  # 忽略 SSL 警告

# 加载网页内容
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/", verify_ssl=False)

# 使用嵌入模型和索引创建器
embedding = OpenAIEmbeddings(api_key="your_openai_api_key")  # 请替换为你的 API 密钥
index_creator = VectorstoreIndexCreator(embedding=embedding)
index = index_creator.from_loaders([loader])

# 测试查询
query = "解释一下文章的主要内容。"
results = index.query(query)
print("查询结果:", results)